<a href="https://colab.research.google.com/github/simecek/dspracticum2020/blob/master/lecture_08/assignment/e_coli_promoters/e_coli_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install biopython

     |████████████████████████████████| 2.3MB 13.6MB/s 


In [2]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Reshaping data from fasta to txt

In [3]:
classes = ['nonpromoters', 'promoters']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://raw.githubusercontent.com/simecek/dspracticum2020/master/lecture_08/assignment/e_coli_promoters/e_coli_{c}_{s}.fa", f"e_coli_{c}_{s}.fa")

In [4]:
for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

In [5]:
for c in classes:
    for s in sets:
        with open(f"e_coli_{c}_{s}.fa", "r") as fr:
            for record in SeqIO.parse(fr, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])


## Reading data

In [6]:
batch_size = 128

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 6791 files belonging to 2 classes.


In [7]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

Found 750 files belonging to 2 classes.


In [8]:
vectorize_layer = TextVectorization(output_mode='int')

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n']))

In [9]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)

## Model training

In [10]:
# one-hot encoding
onehot_layer = keras.layers.Lambda(lambda x: tf.one_hot(tf.cast(x,'int64'), 4))

In [14]:
model_lstm = tf.keras.Sequential([
    onehot_layer,
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.LSTM(128, return_sequences=False),
    keras.layers.Dense(1, activation="sigmoid")]) 

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  

In [15]:
epochs = 10
history = model_lstm.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
54/54 [==============================] - 3s 62ms/step - loss: 0.5915 - accuracy: 0.6768 - val_loss: 0.5393 - val_accuracy: 0.7347
Epoch 2/10
54/54 [==============================] - 3s 51ms/step - loss: 0.4960 - accuracy: 0.7659 - val_loss: 0.4588 - val_accuracy: 0.7840
Epoch 3/10
54/54 [==============================] - 3s 52ms/step - loss: 0.4737 - accuracy: 0.7788 - val_loss: 0.4996 - val_accuracy: 0.7680
Epoch 4/10
54/54 [==============================] - 3s 51ms/step - loss: 0.4452 - accuracy: 0.7989 - val_loss: 0.4251 - val_accuracy: 0.8120
Epoch 5/10
54/54 [==============================] - 3s 51ms/step - loss: 0.4389 - accuracy: 0.8080 - val_loss: 0.4261 - val_accuracy: 0.8147
Epoch 6/10
54/54 [==============================] - 3s 50ms/step - loss: 0.4136 - accuracy: 0.8192 - val_loss: 0.3993 - val_accuracy: 0.8320
Epoch 7/10
54/54 [==============================] - 3s 52ms/step - loss: 0.4138 - accuracy: 0.8193 - val_loss: 0.4129 - val_accuracy: 0.8173
Epoch 8/10
54

In [22]:
model_cnn = tf.keras.Sequential([
    onehot_layer,
    keras.layers.Conv1D(32, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(16, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Conv1D(4, kernel_size=6, data_format='channels_last', activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling1D(),
    keras.layers.Dropout(0.3),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(1, activation="sigmoid")
])

model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
history = model_cnn.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
54/54 [==============================] - 2s 39ms/step - loss: 0.5969 - accuracy: 0.6974 - val_loss: 0.6737 - val_accuracy: 0.5347
Epoch 2/10
54/54 [==============================] - 2s 35ms/step - loss: 0.5224 - accuracy: 0.7682 - val_loss: 0.5926 - val_accuracy: 0.7040
Epoch 3/10
54/54 [==============================] - 2s 35ms/step - loss: 0.4823 - accuracy: 0.7965 - val_loss: 0.6369 - val_accuracy: 0.5693
Epoch 4/10
54/54 [==============================] - 2s 35ms/step - loss: 0.4203 - accuracy: 0.8379 - val_loss: 0.7594 - val_accuracy: 0.5000
Epoch 5/10
54/54 [==============================] - 2s 36ms/step - loss: 0.3600 - accuracy: 0.8717 - val_loss: 0.6618 - val_accuracy: 0.5467
Epoch 6/10
54/54 [==============================] - 2s 38ms/step - loss: 0.3082 - accuracy: 0.8969 - val_loss: 0.5251 - val_accuracy: 0.6987
Epoch 7/10
54/54 [==============================] - 2s 37ms/step - loss: 0.2727 - accuracy: 0.9094 - val_loss: 0.4243 - val_accuracy: 0.8027
Epoch 8/10
54